In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
import pandas as pd
from pathlib import Path
import regex as re
import csv
from itertools import islice
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
    
def pre_process(doc):
    doc = str(doc)
    x = re.sub('[…“”%!&"@#()\-\*\+,/:;<=>?@[\]\^_`{\}~]', ' ', doc)
    y = re.sub('[\.\']', "", x)
    z = y.lower()
    return z

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# def load_answers(path):
#     """
#     dict - docid map to doc text
#     """
#     docs = {}

#     with open(path,'r') as f:
#         for line in f:
#             # [qid, doc_id, rank]
#             line = line.strip().split('\t')

#             if len(line) == 4:
#                 docid = int(line[1])
#                 doc = line[2]

#                 docs[docid] = []
#                 docs[docid].append(doc)
    
#     return docs

# docs = load_ansers(path + "FiQA_train_doc_final.tsv")

In [0]:
path = "drive/My Drive/FiQA/"

def load_answers(path):
    """
    Returns a dataframe of cols: docid, doc
    """
    # Doc ID to Doc text
    collection = pd.read_csv(path, sep="\t")
    collection = collection[['docid', 'doc']]
    collection = collection.sort_values(by=['docid'])

    return collection

collection = load_answers(path + "FiQA_train_doc_final.tsv")

def load_questions(path):
    """
    Returns a dataframe of cols: qid, question
    """
    # Question ID and Question text
    query_df = pd.read_csv(path, sep="\t")
    queries = query_df[['qid', 'question']]

    return queries

queries = load_questions(path + "FiQA_train_question_final.tsv")

# Question ID and Answer ID pair
qid_docid = pd.read_csv(path + "FiQA_train_question_doc_final.tsv", sep="\t")
qid_docid = qid_docid [['qid', 'docid']]

In [6]:
qid_docid.head(5)

,qid,docid
0,0,18850
1,1,14255
2,2,308938
3,3,296717
4,3,100764


In [23]:
dataset = qid_docid

dataset = dataset.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()

dataset.head(5)

,qid,docid
0,0,[18850]
1,1,[14255]
2,2,[308938]
3,3,"[296717, 100764, 314352, 146317]"
4,4,[196463]


In [24]:
len(dataset)

6648

In [58]:
train, test_set = train_test_split(dataset, test_size=0.05)
train_set, valid_set = train_test_split(train, test_size=0.1)

# Number of questions in each set
print(len(train_set))
print(len(test_set))
print(len(valid_set))

5683
333
632


In [59]:
train_data = train_set.explode('docid')
test_data = test_set.explode('docid')
valid_data = valid_set.explode('docid')

print(len(train_data))
print(len(test_data))
print(len(valid_data))

14587
868
1655


In [84]:
train_data.head(10)

,qid,docid
0,0,18850
2,2,308938
3,3,296717
3,3,100764
3,3,314352
3,3,146317
4,4,196463
5,5,69306
6,6,560251
6,6,188530


In [78]:
def label_to_dict(df):
    qid_rel = {}
    for index, row in df.iterrows():
        if row['qid'] not in qid_rel:
            qid_rel[row['qid']] = []
        qid_rel[row['qid']].append(row['docid'])

    return qid_rel

train = label_to_dict(train_data)
test = label_to_dict(test_data)
valid = label_to_dict(valid_data)

take(5, test.items())

[(7, [411063]),
 (9, [509122, 184698]),
 (86, [71569]),
 (98, [575929, 527522]),
 (110, [520386, 220063, 410128, 459119, 482165, 397152])]

In [0]:
with open(path + 'qid_rel_train.pickle', 'wb') as handle:
    pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path + 'qid_rel_test.pickle', 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path + 'qid_rel_valid.pickle', 'wb') as handle:
    pickle.dump(valid, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [82]:
train_q = list(train.keys())
test_q = list(test.keys())
valid_q = list(valid.keys())

print(len(train_q))
print(len(test_q))
print(len(valid_q))

5683
333
632


In [86]:
train_questions = queries[queries['qid'].isin(train_q)]
test_questions = queries[queries['qid'].isin(test_q)]
valid_questions = queries[queries['qid'].isin(valid_q)]

,qid,question
1,1,Claiming business expenses for a business with...
8,8,How to deposit a cheque issued to an associate...
11,12,Does U.S. tax code call for small business own...
20,21,Deducting last years (undocumented) side busin...
22,25,Claiming business expense from personal credit...


In [87]:
print(len(train_questions))
print(len(test_questions))
print(len(valid_questions))

5683
333
632


In [0]:
with open(path + "train_questions.tsv",'w') as write_tsv:
    write_tsv.write(train_questions.to_csv(sep='\t', index=False, header=False))

with open(path + "test_questions.tsv",'w') as write_tsv:
    write_tsv.write(test_questions.to_csv(sep='\t', index=False, header=False))

with open(path + "valid_questions.tsv",'w') as write_tsv:
    write_tsv.write(valid_questions.to_csv(sep='\t', index=False, header=False))

In [3]:


# Cleaning data
empty_docs = []
empty_id = []

for index, row in collection.iterrows():
    if pd.isna(row['doc']):
        empty_docs.append(row['docid'])
        empty_id.append(index)

# print(empty_docs)
print(empty_id)

# Remove empty answers
collection2 = collection.drop(empty_id)

# Answers Tokenization
collection2['doc_processed'] = collection2['doc'].apply(pre_process)
collection2['tokenized_ans'] = collection2.apply(lambda row: wordpunct_tokenize(row['doc_processed']), axis=1)
collection2['ans_len'] = collection2.apply(lambda row: len(row['tokenized_ans']), axis=1)

collection2.head(5)

len(collection2)

avg_ans_count = collection2['ans_len'].mean()

print(avg_ans_count)

# Questions Tokenization
queries = queries.copy()
queries['q_processed'] = queries['question'].apply(pre_process)
queries['tokenized_q'] = queries.apply(lambda row: wordpunct_tokenize(row['q_processed']), axis=1)
queries['q_len'] = queries.apply(lambda row: len(row['tokenized_q']), axis=1)

queries.head(5)

avg_q_count = queries['q_len'].mean()

print(avg_q_count)

# collection2.to_csv("processed_ans.csv", index=None)
# queries.to_csv("processed_q.csv", index=None)

[741, 1158, 1329, 3208, 3937, 5266, 6399, 11237, 11407, 12328, 13459, 14869, 16143, 16326, 18297, 19839, 19944, 20739, 22923, 23949, 24603, 28093, 30966, 31492, 34386, 34539, 34697, 38382, 39802, 42403, 43008, 45178, 46794, 50845, 53111, 54963, 56460, 57452]


KeyboardInterrupt: ignored

In [0]:
print(empty_docs)

[7915, 12229, 14135, 33445, 40982, 54960, 66248, 115636, 117276, 126502, 138418, 153104, 167128, 169220, 189754, 206368, 207473, 215635, 237392, 248226, 254541, 290110, 319894, 325407, 356535, 358213, 360125, 399083, 414219, 441462, 447488, 470232, 486245, 528558, 552533, 572451, 587667, 597929]


In [0]:
len(collection2)

57600

In [0]:
with open('empty_docs.pickle', 'wb') as handle:
    pickle.dump(empty_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# word2index = {"PAD": 0}
# word2count = {}

# idx = 1

# for index, row in collection2.iterrows():
#     for word in row['tokenized_ans']:
#         if word not in word2index:
#             word2index[word] = idx
#             idx += 1
#             word2count[word] = 1
#         else:
#             word2count[word] += 1
            
# ans_vocab_size = len(word2index)

# print("Answer vocab size: {}".format(ans_vocab_size))

# idx = len(word2index)

# for index, row in queries.iterrows():
#     for word in row['tokenized_q']:
#         if word not in word2index:
#             word2index[word] = idx
#             idx += 1
#             word2count[word] = 1
#         else:
#             word2count[word] += 1

# print(len(word2index))

# q_vocab_size = len(word2index) - ans_vocab_size
# print("Question vocab size: {}".format(q_vocab_size))

In [0]:
# # Reduce the size of the vocabuary
# word2idx = {"PAD": 0}
# word2c = {}
# idx = 1

# for word, count in word2count.items():
#     if count > 3:
#         if word not in word2idx:
#             word2idx[word] = idx
#             idx += 1
#             word2c[word] = count

# print(word2idx)

In [0]:
# # Vocab size
# len(word2idx)

# c = Counter(word2c)
# mc = c.most_common(30)

In [0]:
# with open('vocab_full.pickle', 'wb') as handle:
#     pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('vocab.pickle', 'wb') as handle:
#     pickle.dump(word2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('vocab_count.pickle', 'wb') as handle:
#     pickle.dump(word2c, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # Label to answer text
# docid_tokenized = collection2[['docid', 'tokenized_ans']]

# docid_tokenized.head(5)

# label_to_ans = {}

# for index, row in docid_tokenized.iterrows():
#     label_to_ans[row['docid']] = row['tokenized_ans']

# print(take(5, label_to_ans.items()))

# with open('label_ans.pickle', 'wb') as handle:
#     pickle.dump(label_to_ans, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# # Question to question text
# q_tokenized = queries[['qid', 'tokenized_q']]

# qid_to_text = {}

# for index, row in q_tokenized.iterrows():
#     qid_to_text[row['qid']] = row['tokenized_q']

# print(take(5, qid_to_text.items()))

# with open('qid_text.pickle', 'wb') as handle:
#     pickle.dump(qid_to_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Answer Ranking for each question
doc_ranking = pd.read_csv(path + "run_train_small.tsv", sep="\t", header=None)
doc_ranking = doc_ranking.rename(columns={0: 'qid', 1: 'doc_id', 3:'rank'})

# Create dict for query id and ranked candidates
# key: query ids, values: list of 1000 ranked candidates
qid_ranked_docs = {}

with open(path + "run_train_small.tsv",'r') as f:
    for line in f:
        # [qid, doc_id, rank]
        line = line.strip().split('\t')
        qid = int(line[0])
        doc_id = int(line[1])
        rank = int(line[2])
        
        if qid not in qid_ranked_docs:
            # Create a list of size 1000 for each query to store the candidates
            candidates = [0]*100
            qid_ranked_docs[qid] = candidates
        qid_ranked_docs[qid][rank-1] = doc_id
        
print(take(1, qid_ranked_docs.items()))

save_pickle('qid_ranked_docs_100.pickle', qid_ranked_docs)

In [0]:
# qid_rel = {}

# for index, row in qid_docid.iterrows():
    
#     if row['qid'] not in qid_rel:
#         qid_rel[row['qid']] = []
#     qid_rel[row['qid']].append(row['docid'])
    
# with open('qid_rel.pickle', 'wb') as handle:
#     pickle.dump(qid_rel, handle, protocol=pickle.HIGHEST_PROTOCOL)